This notebook will contain some scratch work on searching the ASTs of some julia code looking for reasoning about types. I'll probably also have some plots in here.

# Setup

In [1]:
addprocs(6)

6-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7

In [1]:
push!(LOAD_PATH, pwd());

In [ ]:
using C
using ASTp
using DataFrames
import DataFrames.head
using FileTrees
using DataStructures
import Base.return_types

In [ ]:
reload("C")
reload("ASTp")

In [ ]:
test_dir = "test_modules/"
@everywhere eco_dir = "/Users/isaac/Documents/julia_repos/"
base_dir = "/Users/isaac/github/julia/Base/"
test_files = C.search_dirs(test_dir)
eco_files = C.search_dirs(eco_dir)
base_files = C.search_dirs(base_dir);

# Examples

### methods using methods that use Types

In [4]:
@time begin
    @everywhere type_mt = methodswith(Type, Base);
    @everywhere type_funcs = unique(map(x->x.func.name::Symbol, type_mt));
end

  1.625210 seconds (933.35 k allocations: 36.952 MB, 0.55% gc time)


In [5]:
# Getting signatures for methods
# To interact with the signatures, use `sig.parameters`
@everywhere tmethod_info = let d = DataStructures.OrderedDict{Symbol,Dict}([t=>Dict{Any,Any}("sigs" => []) for t in type_funcs]...)
    for m in type_mt
        push!(d[m.func.name]["sigs"], m.sig)
    end
    for v in values(d)
        v["siglengths"] = unique(Int[length(s.parameters) for s in v["sigs"]])
    end
    d
end
    

In [6]:
# parse_ast(test_files, 
# )
@everywhere function process_file(path::AbstractString, s::C.Selector)
#     println(path)
    ast = C.parse_file(path)
    exprs_list = try 
        C.parse_ast(ast, s)
    catch x
        println(path)
        println(x)
        return [x]
    end
    return exprs_list
#   count_fields(exprs_list)
end

In [7]:
# # @everywhere function gettypefunctions(files, degree::Int)
# @everywhere function gettypefunctions(degree::Int, comparison=(==))
#         C.Selector(Any[
#             ASTp.isfunction, # Only look at functions
#             [
#                 ASTp.functionbody, # Inside their guts
#                 y->C.parse_ast(y,C.Selector([ # Count the number of functions they use which can be dispatched on Types
#                     x->isa(x,Expr),
#                     x->ASTp.iscalling(x,type_funcs)
#                     ])),
# #                 x->length(x)>degree # Filter by count of functions
#             x->comparison(length(x),degree)
#             ]
#         ]
#     )#)
# end

In [8]:
# @everywhere function gettypefunctions(files, degree::Int)
@everywhere function gettypefunctions(degree::Int, comparison=(==))
        C.Selector(Any[
            ASTp.isfunction,# Only look at functions
        x->!ASTp.isanon(x), # Don't want anonymous functions. Only declared ones
            [
                ASTp.functionbody, # Inside their guts
                y->C.parse_ast(y,C.Selector([ # Count the number of functions they use which can be dispatched on Types
                    x->isa(x,Expr),
                    x->ASTp.iscalling(x,type_funcs),
                    x->length(x.args) in  tmethod_info[x.args[1]]["siglengths"] # Signatures should at least be of right length
                    ])),
#                 x->length(x)>degree # Filter by count of functions
            x->comparison(length(x),degree)
            ]
        ]
    )#)
end

In [9]:
@everywhere function get_counts(file)
    data = map(1:10) do number
        f = gettypefunctions(number)
        process_file(file, f)
    end
#     println(typeof(data))
#     println(typeof(gettypefunctions(10, >)))
#     append!(data, gettypefunctions(gettypefunctions(10, >)))
    return data
end




### I want to figure out input -> output types for some functions

* So I can use built in stuff.
* However, I'll need to know how to call things, so I'll want a mapping of functions to the module they were defined in.
* So I'll also need to be able to work with nested modules.

`module_tree` has been created


#### Issues

* `<|` is exported, but I can't find any definition being assigned to it

### return type inference on a method

In [10]:
fieldsntypes(f) = map(x->(x, fieldtype(typeof(f), x)), fieldnames(f))

fieldsntypes (generic function with 1 method)

In [10]:
fieldsntypes(temp2)

LoadError: LoadError: UndefVarError: temp2 not defined
while loading In[10], in expression starting on line 1

In [11]:
"""
Trys to infer which types a method could return.
"""
function return_types(m::TypeMapEntry)
    linfo = m.func
    atypes = Base.to_tuple_type(m.sig)
    if !isa(m.tvars, SimpleVector)
        sparams = Base.svec(m.tvars)
    else
        sparams = m.tvars
    end
    @assert isa(sparams, SimpleVector)
    # I think I need to get m.tvars into a SimpleVector
    (_li, ty) = Core.Inference.typeinf(linfo, atypes, sparams) # Retuns `(lowered, return_type)`
    ty
end
return_types(type_mt[end-1])


Union{Base.RefValue{T},Ptr{T}}

In [18]:
"""
Counts how many time a method calls it's generic function. 
"""
function same_function(x)
    name = functionname(x)
    body = functionbody(x)
    parse_ast(body, Selector([isexpr, iscalling(name)]))
end

same_function

### TODO: Collect all functions which return types

### Mappings of whether names need to be qualified/ how much

In [ ]:
names(Base)
names()

# Filtering raw data

I pick up some weird stuff in the raw data. Here I'll be establishing what I'm clearing out.

In [19]:
"""Function to be applied to each element of list of function expressions to see if they could be filtered."""
function filter_expr(expr::Expr)
    if isa(expr.args[1], Bool) # Some weird stuff in Markdown with `@breaking` macro
        return false
    end
    return true
end 
filter_expr(not_expr::Any) = false

filter_expr (generic function with 2 methods)

# Take 2: Finding a better way to organize my data

Here I am going to try and keep the data around. Basically I want to have a format where it's easy for me to interogate the data

Right now a dataframe in long format is looking appealing, as I can just groupby

In [14]:
raw = pmap(base_files) do file
    process_file(file, gettypefunctions(1, >))
end;
raw = pmap(x->filter(filter_expr, x), raw)

/Users/isaac/github/julia/Base/atomics.jl
BoundsError(Any[:atomic_cas!],(2,))
/Users/isaac/github/julia/Base/boot.jl
BoundsError(Any[:Typeof],(2,))
/Users/isaac/github/julia/Base/checked.jl
BoundsError(Any[:checked_neg],(2,))
/Users/isaac/github/julia/Base/dates/adjusters.jl
BoundsError(Any[:firstdayofweek],(2,))
/Users/isaac/github/julia/Base/dates/periods.jl
BoundsError(Any[:default],(2,))
/Users/isaac/github/julia/Base/deprecated.jl
BoundsError(Any[:msync],(2,))
/Users/isaac/github/julia/Base/int.jl
BoundsError(Any[:abs],(2,))
/Users/isaac/github/julia/Base/io.jl
BoundsError(Any[:isopen],(2,))
/Users/isaac/github/julia/Base/libc.jl
BoundsError(Any[:GetLastError],(2,))
/Users/isaac/github/julia/Base/libdl.jl
BoundsError(Any[:dlopen],(2,))
/Users/isaac/github/julia/Base/linalg/blas.jl
BoundsError(Any[:blascopy!],(2,))
/Users/isaac/github/julia/Base/profile.jl
BoundsError(Any[:callers],(2,))
/Users/isaac/github/julia/Base/reflection.jl
BoundsError(Any[:instances],(2,))
/Users/isaac/git

288-element Array{Any,1}:
 Any[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [15]:
rmprocs(workers())

:ok

In [7]:
using DataFrames
import DataFrames.head
using JLD

    size(DataFrames.ModelMatrix, Any...) at /Users/isaac/.julia/v0.5/DataFrames/src/statsmodels/formula.jl:48
is ambiguous with: 
    size(Any, Integer, Integer, Integer...) at abstractarray.jl:23.
To fix, define 
    size(DataFrames.ModelMatrix, Integer, Integer, Integer...)
before the new definition.
    size(DataFrames.ModelMatrix, Any...) at /Users/isaac/.julia/v0.5/DataFrames/src/statsmodels/formula.jl:48
is ambiguous with: 
    size(Any, Integer, Integer, Integer...) at abstractarray.jl:23.
To fix, define 
    size(DataFrames.ModelMatrix, Integer, Integer, Integer...)
before the new definition.
    size(DataFrames.ModelMatrix, Any...) at /Users/isaac/.julia/v0.5/DataFrames/src/statsmodels/formula.jl:48
is ambiguous with: 
    size(Any, Integer, Integer, Integer...) at abstractarray.jl:23.
To fix, define 
    size(DataFrames.ModelMatrix, Integer, Integer, Integer...)
before the new definition.
    size(DataFrames.ModelMatrix, Any...) at /Users/isaac/.julia/v0.5/DataFrames/src/stat

In [17]:
df = DataFrame()
df[:file] = convert(Array{ASCIIString}, base_files)
df[:exprs] = raw;
sort!(df, cols = order(:exprs, by=length), rev=true);
df[:file] = map(x->x[32:end], df[:file].data);


In [18]:
collect_calls(x) = map(field([:args, 1]), parse_ast(x, Selector([isexpr, iscall])))
"""Literally copy pasted from stack overflow, might sort a counter? select says it will partially sort."""
most_common(c::Accumulator)    = most_common(c, length(c))
most_common(c::Accumulator, k) = select!(collect(c), 1:k, by=kv->kv[2], rev=true)

most_common (generic function with 2 methods)

## Expanding DataFrame by expression

* Each expression gets it's own row
* **Names** of functions defined by expression are also extracted

In [37]:
"""Extract function names from expressions"""
exprnames(exprs) = map(x->try ASTp.functionname(x) catch nothing end, exprs) # Definitions with strange syntax.

exprnames

In [20]:
# Expanding dataframe by expression
exdf = by(df, :file) do df
    n = length(df[1, :exprs])
    f = df[1, :file]
    DataFrame(expr=df[1, :exprs])
end
# Filtering out bounds errors from earlier
exdf = exdf[![isa(x,BoundsError) for x in exdf[:expr]],:]
exdf[:expr] = convert(Array{Expr,1}, exdf[:expr].data) # Now we have a single type.;
exdf[:name] = exprnames(exdf[:expr]); # Has errors, probably shouldn't index on this.;

In [38]:
# Getting most common functions for the files in the expressions.
let tempdf = deepcopy(exdf)
    tempdf[:functions] = map(collect_calls, tempdf[:expr])
    by(tempdf, :file) do subdf
        f = subdf[1, :file]
        fs = vcat(subdf[:functions]...)
        temp = filter(x->x in type_funcs, fs) |> counter |> most_common#x->x.map |> collect
#         DataFrame(counts = counter() |> most_common)
        DataFrame(subdf[:])
    end
end

,file,file_1,expr,name,functions
1,Enums.jl,Enums.jl,"function membershiptest(expr,values) # none, line 17: (lo,hi) = extrema(values) # none, line 18: if length(values) == (hi - lo) + 1 # none, line 19: $(Expr(:quote, :($(Expr(:$, :lo)) <= $(Expr(:$, :expr)) <= $(Expr(:$, :hi))))) else # none, line 20: if length(values) < 20 # none, line 21: foldl(((x1,x2)->begin # none, line 21: $(Expr(:quote, :($(Expr(:$, :x1)) || $(Expr(:$, :expr)) == $(Expr(:$, :x2))))) end),$(Expr(:quote, :($(Expr(:$, :expr)) == $(Expr(:$, :(values[1])))))),values[2:end]) else # none, line 23: $(Expr(:quote, :($(Expr(:$, :expr)) in $(Expr(:$, :(Set(values))))))) end end end",membershiptest,"Any[:membershiptest,:extrema,:(==),:length,:+,:-,:<,:length,:foldl,:(==),:(==),:in,:Set]"
2,LineEdit.jl,LineEdit.jl,"function show_completions(s::PromptState,completions) # none, line 118: colmax = maximum(map(length,completions)) # none, line 119: num_cols = max(div(width(terminal(s)),colmax + 2),1) # none, line 120: (entries_per_col,r) = divrem(length(completions),num_cols) # none, line 121: entries_per_col += r != 0 # none, line 123: cmove_down(terminal(s),input_string_newlines_aftercursor(s)) # none, line 124: println(terminal(s)) # none, line 125: for row = 1:entries_per_col # none, line 126: for col = 0:num_cols # none, line 127: idx = row + col * entries_per_col # none, line 128: if idx <= length(completions) # none, line 129: cmove_col(terminal(s),(colmax + 2) * col) # none, line 130: print(terminal(s),completions[idx]) end end # none, line 133: println(terminal(s)) end # none, line 136: for i = 1:input_string_newlines(s) # none, line 137: println(terminal(s)) end end",show_completions,"Any[:show_completions,:maximum,:map,:max,:div,:width,:terminal,:+,:divrem,:length,:(!=),:cmove_down,:terminal,:input_string_newlines_aftercursor,:println,:terminal,:+,:*,:(<=),:length,:cmove_col,:terminal,:*,:+,:print,:terminal,:println,:terminal,:input_string_newlines,:println,:terminal]"
3,LineEdit.jl,LineEdit.jl,"function complete_line(s::PromptState,repeats) # none, line 144: (completions,partial,should_complete) = complete_line(s.p.complete,s) # none, line 145: if isempty(completions) # none, line 146: beep(terminal(s)) else # none, line 147: if !should_complete # none, line 150: show_completions(s,completions) else # none, line 151: if length(completions) == 1 # none, line 153: prev_pos = position(s.input_buffer) # none, line 154: seek(s.input_buffer,prev_pos - sizeof(partial)) # none, line 155: edit_replace(s,position(s.input_buffer),prev_pos,completions[1]) else # none, line 157: p = common_prefix(completions) # none, line 158: if !(isempty(p)) && p != partial # none, line 161: prev_pos = position(s.input_buffer) # none, line 162: seek(s.input_buffer,prev_pos - sizeof(partial)) # none, line 163: edit_replace(s,position(s.input_buffer),prev_pos,p) else # none, line 164: if repeats > 0 # none, line 165: show_completions(s,completions) end end end end end end",complete_line,"Any[:complete_line,:complete_line,:isempty,:beep,:terminal,:!,:show_completions,:(==),:length,:position,:seek,:-,:sizeof,:edit_replace,:position,:common_prefix,:!,:isempty,:(!=),:position,:seek,:-,:sizeof,:edit_replace,:position,:>,:show_completions]"
4,LineEdit.jl,LineEdit.jl,"function refresh_multi_line(termbuf::TerminalBuffer,terminal::UnixTerminal,buf,state::InputAreaState,prompt=""""; indent=0) # none, line 195: _clear_input_area(termbuf,state) # none, line 197: cols = width(terminal) # none, line 198: curs_row = -1 # none, line 199: curs_pos = -1 # none, line 200: cur_row = 0 # none, line 201: buf_pos = position(buf) # none, line 202: line_pos = buf_pos # none, line 204: write_prompt(termbuf,prompt) # none, line 205: prompt = prompt_string(prompt) # none, line 207: miscountnl = @windows(?,(isa(Terminals.pipe_reader(terminal),Base.TTY) && !(Base.ispty(Terminals.pipe_reader(terminal)))):false) # none, line 208: lindent = strwidth(prompt) # none, line 211: seek(buf,0) # none, line 212: moreinput = tr

In [39]:
# Most common over all
let temp = map(collect_calls, exdf[:expr])
    temp = vcat(temp...)
    filter(x->x in type_funcs, temp) |> counter |> most_common |> Base.showarray
end

66-element Array{Any,1}:
 Pair{Any,Int64}(:(==),2550)       
 Pair{Any,Int64}(:length,1532)     
 Pair{Any,Int64}(:(!=),759)        
 Pair{Any,Int64}(:convert,459)     
 Pair{Any,Int64}(:zero,448)        
 Pair{Any,Int64}(:one,281)         
 Pair{Any,Int64}(:real,238)        
 Pair{Any,Int64}(:similar,229)     
 Pair{Any,Int64}(:eltype,191)      
 Pair{Any,Int64}(:zeros,139)       
 Pair{Any,Int64}(:promote_type,132)
 Pair{Any,Int64}(:error,120)       
 Pair{Any,Int64}(:oftype,100)      
 Pair{Any,Int64}(:reinterpret,99)  
 Pair{Any,Int64}(:typemax,83)      
 Pair{Any,Int64}(:float,81)        
 Pair{Any,Int64}(:map,67)          
 Pair{Any,Int64}(:ndims,62)        
 Pair{Any,Int64}(:read,61)         
 Pair{Any,Int64}(:complex,56)      
 Pair{Any,Int64}(:rand,51)         
 Pair{Any,Int64}(:parse,44)        
 Pair{Any,Int64}(:typemin,42)      
 Pair{Any,Int64}(:round,39)        
 Pair{Any,Int64}(:checkbounds,38)  
 Pair{Any,Int64}(:deserialize,35)  
 Pair{Any,Int64}(:isbits,31)       
 Pa

In [40]:
let tempdf = deepcopy(exdf)
    tempdf[:method_calls] = map(collect_calls, exdf[:expr])
    by(exdf, :file) do subdf
        counter(vcat(subdf[:method_calls]...))
        

LoadError: LoadError: syntax: incomplete: "do" at In[40]:3 requires end
while loading In[40], in expression starting on line 1

In [41]:
callsdf = by(exdf, :expr) do subdf
    name = subdf[1,:name]
    file = subdf[1, :file]
    expr = subdf[1,:expr]
    sig = ASTp.functionhead(expr)
    calls = collect_calls(subdf[1,:expr]) |> counter |> most_common
    call_name = map(x->x[1], calls)
    call_count = map(x->x[2], calls)
    idxs = findin(call_name, type_funcs)
    call_name = call_name[idxs]
    call_count = call_count[idxs]
    n = length(call_name)
    DataFrame(file=fill(file,n) , name=fill(name,n), sig=fill(sig,n), called=call_name, count=call_count)
end;

In [42]:
# Being able to infer dynamic information, types in -> types out
temp = by(callsdf, :expr) do subdf
    # TODO some values give multiple expressions
    name = subdf[1,:name] 
    file = subdf[1,:file] 
    sig = subdf[1,:sig] # So I can figure out which methods
    DataFrame(file=file, name=name, sig=sig, count=sum(subdf[:count]))
end |> x->sort(x, cols=:count, rev=true)

1942x5 DataFrames.DataFrame
│ Row  │ ┝━━━━━━┿
│ 1    │ │ 2    │ │ 3    │ │ 4    │ │ 5    │ │ 6    │ │ 7    │ │ 8    │ │ 9    │ │ 10   │ │ 11   │ 
⋮
│ 1931 │ │ 1932 │ │ 1933 │ │ 1934 │ │ 1935 │ │ 1936 │ │ 1937 │ │ 1938 │ │ 1939 │ │ 1940 │ │ 1941 │ │ 1942 │ 

│ Row  │ expr                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# Trying to write data out to R so I can open it in excel to get a better look at it

In [ ]:
using RCall

In [ ]:
temp[:expr] = map(string, temp[:expr])
temp[:name] = map(string, temp[:name])
temp[:sig] = map(string, temp[:sig]);

In [ ]:
g = globalEnv
reval(rparse("dfls <- NULL"))

In [ ]:
rrun(x::AbstractString) = reval(rparse(x))

In [ ]:
rrun("dfls <- NULL")

In [ ]:
for cnm in DataFrames._names(temp)
    g[:colcnm] = RCall.sexp(convert(Array, temp[cnm]))
    reval(rparse("dfls\$$cnm <- colcnm")) # Make a R list
end

In [ ]:
rrun("df <- data.frame(dfls)");

In [ ]:
rrun("library(WriteXLS)")

In [ ]:
rrun("""WriteXLS(df, "test.xls")""")

In [ ]:
rrun("save(file='dfjulia.RData', df)")

# Writing out data

In [ ]:
jldopen("data/plotting/df.jld", "w") do f
    write(f, "df", temp[[:file,:name,:sig,:called,:count]])
end

# TODO

* Make a taxonomy of julia asts that will reason about types
    * With identifying methods
        * These could then be nested to find more precise things
* Read that Julia paper.


### `methodsreturning`

Find all methods which return a certain type

#### Suddenly:

It's very slow. It's something to do with the comparisons being made, since just travelling though the table in the mew way downs seem to do anything. 

Maybe it just takes a really long time to compile?

In [100]:
temp = Set()
function methodsreturning(t::Type, f::Function, showparents::Bool=false, meths = TypeMapEntry[])
    mt = typeof(f).name.mt
    Base.visit(mt) do d
        if (x -> (Base.type_close_enough(x, t) ||
                     (showparents ? (t <: x && (!isa(x,TypeVar) || x.ub != Any)) :
                      (isa(x,TypeVar) && x.ub != Any && t == x.ub)) &&
            x != Any && x != ANY))(return_types(d))
            push!(meths, d)
        end
        push!(temp, typeof(return_types(d)))
    end
    return meths
end

function methodsreturning(t::Type, m::Module, showparents::Bool=false)
    meths = TypeMapEntry[]
    for nm in names(m)
        if isdefined(m, nm)
            f = getfield(m, nm)
            if isa(f, Function)
                methodsreturning(t, f, showparents, meths)
            end
        end
    end
    return unique(meths)
end

function methodsreturning(t::Type, showparents::Bool=false)
    meths = TypeMapEntry[]
    mainmod = current_module()
    # find modules in Main
    for nm in names(mainmod)
        if isdefined(mainmod,nm)
            mod = getfield(mainmod, nm)
            if isa(mod, Module)
                append!(meths, methodsreturning(t, mod, showparents))
            end
        end
    end
    return unique(meths)
end

methodsreturning (generic function with 7 methods)

In [102]:
@time methodsreturning(Int, foo)

  0.000049 seconds (10 allocations: 416 bytes)


1-element Array{TypeMapEntry,1}:
 foo(x::Int64) at In[94]:1

In [104]:
temp = Set()
@time methodsreturning(Type, Main);
# With d.next
temp


  0.016194 seconds (13.58 k allocations: 608.545 KB)


Set(Any[DataType])

In [92]:
@which methodswith(Type)

methodswith(t::Type{T<:Any}) at interactiveutil.jl:414

In [78]:
# temp=0
# # Without d.next
# @time methodsreturning(Type);
# temp

  0.004131 seconds (27.34 k allocations: 483.031 KB)


10251

In [94]:
foo(x::Int) = x::Int
foo(1)
type Foo
end
bar() = Foo()
bar(x::Foo) = x
bar2() = Foo

bar2 (generic function with 1 method)

In [95]:
@time methodswith(Type)

  0.062218 seconds (41.41 k allocations: 1.086 MB)


1350-element Array{TypeMapEntry,1}:
 !=(T::Type{T<:Any}, S::Type{T<:Any}) at operators.jl:27                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 rem(x::BigInt, ::Type{Bool}) at gmp.jl:172                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 rem{T<:Union{Signed,Unsigned}}(x::BigInt, ::Type{T}) at gmp.jl:174                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 rem(x::Bool, ::Type{Int8}) at int.jl:211                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
 rem(x::Bool, ::Type{Int16}) at int.jl:211                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 rem(x::Bool, ::Type{Int32}) at int.jl:211                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 rem(x::Bool, ::Type{Int64}) at int.jl:211                                                                                                                                                                                                                                                                                                                                                                

In [37]:
@time methodsreturning(Type)

LoadError: LoadError: type TypeMapLevel has no field next
while loading In[37], in expression starting on line 155

In [71]:
@time methodswith(Type)

  0.266660 seconds (41.68 k allocations: 1.747 MB)


1519-element Array{Method,1}:
 iscontiguous{F<:SubArray{T,N,P,I<:Tuple{Union{Colon,UnitRange{T<:Real}},Vararg{Any}},true}}(::Type{F}) at subarray.jl:233                                                                                                                                                                                         
 iscontiguous{S<:SubArray{T,N,P,I,L}}(::Type{S}) at subarray.jl:232                                                                                                                                                                                                                                                
 !=(T::Type{T<:Any}, S::Type{T<:Any}) at operators.jl:27                                                                                                                                                                                                                                                           
 rem(x::Int16, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                         
 rem(x::Int32, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                         
 rem(x::Int64, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                         
 rem(x::Int128, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                        
 rem(x::UInt8, ::Type{Int8}) at int.jl:220                                                                                                                                                                                                                                                                         
 rem(x::UInt16, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                        
 rem(x::UInt32, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                        
 rem(x::UInt64, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                        
 rem(x::UInt128, ::Type{Int8}) at int.jl:198                                                                                                                                                                                                                                                                       
 rem(x::Bool, ::Type{Int8}) at int.jl:211                                                                                                                                                                                                                                         

In [ ]:

let d = typeof(foo).name.mt.defs
    t = Int
    meths = []
    showparents = true
    while d !== nothing
        push!(meths, d.func.rettype) # I could probably just dump this and check it with a regular expression.
        
#         if (x -> (Base.type_close_enough(x, t) ||
#                     (showparents ? (t <: x && (!isa(x,TypeVar) || x.ub != Any)) :
#                       (isa(x,TypeVar) && x.ub != Any && t == x.ub)) &&
#                      x != Any && x != ANY))(d.func.rettype)
#             push!(meths, d)
#         end
        d = d.next
    end
    println(meths)
end

In [ ]:
methodsreturning(Type, rem) 

In [ ]:
@time methodswith(Function);

In [ ]:
@time methodswith2(Type)

# Searching eco dirs

In [ ]:
using DataFrames
using FileTrees


In [ ]:
println("hi")

In [7]:
repos = DataFrame()
# repos[:name] = readdir(eco_dir)[2:end];
repos[:name] = readdir(eco_dir) # Smaller sample
repos = by(repos, :name) do subdf
    name = subdf[1, :name]
    dir = joinpath(eco_dir, name)
    head_file = joinpath(dir, "src", string(name, ".jl"))
    DataFrame(dir=dir, head=head_file)
end
let idx = find(x->x=="React", repos[:name].data) # React got named weird
    repos[idx, :head] = joinpath(eco_dir, "React", "src", "Reactive.jl")
end;
# deleterows!(repos, find(x->x=="Hwloc", repos[:name].data)) # Wants to be built to have all files;

In [ ]:
temp = by(repos, :name) do subdf
    out = map(subdf[:files].data) do pth
        println(pth)
        ast = parse_file(pth)
        results = parse_ast(ast, Selector([isexpr, istypedecl]))
        println(size(results))
        results
    end
    out = vcat(out)
    println(size(out), typeof(out))
    out
end

/Users/isaac/Documents/julia_repos/ASCIIPlots/src/imagesc.jl
(0,)
/Users/isaac/Documents/julia_repos/ASCIIPlots/src/lineplot.jl
(0,)
/Users/isaac/Documents/julia_repos/ASCIIPlots/src/scatterplot.jl
(0,)
(3,)Array{Any,1}
/Users/isaac/Documents/julia_repos/BSplines/src/code.jl
(0,)
(1,)Array{Any,1}
/Users/isaac/Documents/julia_repos/BlsData/src/get_data.jl
(0,)
(1,)Array{Any,1}
/Users/isaac/Documents/julia_repos/CUDA/src/errors.jl
(1,)
/Users/isaac/Documents/julia_repos/CUDA/src/funmap.jl
((0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/fisher.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/generic.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/gmm.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/heap-def.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/hikmeans.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/hog.jl
(0,)
/Users/isaac/Documents/julia_repos/VLFeat/src/libvl/homkermap.jl
(0,)
/Users/isaac/Documents/julia_re

,name,x1
1,ASCIIPlots,Any[]
2,ASCIIPlots,Any[]
3,ASCIIPlots,Any[]
4,BSplines,Any[]
5,BlsData,Any[]
6,CUDA,"Any[:(immutable CuDriverError # none, line 4: code::Int end)]"
7,CUDA,Any[]
8,CUDA,Any[]
9,CUDA,"Any[:(immutable CuDevice # none, line 4: ordinal::Cint # none, line 5: handle::Cint # none, line 7: function CuDevice(i::Int) # none, line 8: ordinal = convert(Cint,i) # none, line 9: a = Ref{Cint}() # none, line 10: @cucall cuDeviceGet (Ptr{Cint},Cint) a ordinal # none, line 11: handle = a[] # none, line 12: new(ordinal,handle) end end),:(immutable CuCapability # none, line 3: major::Int # none, line 4: minor::Int end)]"
10,CUDA,"Any[:(immutable CuContext # none, line 4: handle::Ptr{Void} end)]"


In [ ]:
![true, false]

# Make a directed graph of usage between all packages.

In [7]:
repos = DataFrame(names = Pkg.available())
repos[:url] = [Pkg.Read.url(name) for name in repos[:names]]
repos[:dir] = map(x->joinpath(eco_dir, x), repos[:names]);

In [8]:
addprocs(6)
using C
using DataOps

    (::Type{DataStructures.HashDict{#K<:Any, #V<:Any, #O<:Union{Int64, Void}}})(Any, Any) at /Users/isaac/.julia/v0.5/DataStructures/src/hash_dict.jl:40
is ambiguous with: 
    (::Type{DataStructures.HashDict{#K<:Any, #V<:Any, #O<:Union{Int64, Void}}})(Base.Pair{A<:Any, B<:Any}...) at /Users/isaac/.julia/v0.5/DataStructures/src/hash_dict.jl:31.
To fix, define 
    (::Type{DataStructures.HashDict{#K<:Any, #V<:Any, #O<:Union{Int64, Void}}})(Base.Pair{A<:Any, B<:Any}, Base.Pair{A<:Any, B<:Any})
before the new definition.
    (::Type{DataStructures.HashDict{#K<:Any, #V<:Any, #O<:Union{Int64, Void}}})(Any, Any) at /Users/isaac/.julia/v0.5/DataStructures/src/hash_dict.jl:40
is ambiguous with: 
    (::Type{DataStructures.HashDict{#K<:Any, #V<:Any, #O<:Union{Int64, Void}}})(Base.Pair{A<:Any, B<:Any}...) at /Users/isaac/.julia/v0.5/DataStructures/src/hash_dict.jl:31.
To fix, define 
    (::Type{DataStructures.HashDict{#K<:Any, #V<:WARNING: New definition 
    (::Type{DataStructures.HashDict{#K<

In [8]:
@everywhere pkgs = Pkg.available()
@everywhere module_usage_closure(dir) = x->DataOps.module_usage(dir, x)

In [ ]:
usage = pmap(pkgs) do p
    println(p)
    dir = joinpath(eco_dir, p)
    map(module_usage_closure(dir), pkgs)
end

  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely near In[14]:1
  likely ne

	From worker 7:	CUDNN
	From worker 4:	DriftDiffusionPoissonSystems
	From worker 3:	Bezier
	From worker 6:	Clustering
	From worker 2:	AbstractDomains
	From worker 5:	FixedEffectModels
	From worker 7:	CUFFT
	From worker 3:	BIGUQ
	From worker 5:	FixedPointNumbers
	From worker 2:	AbstractTrees
	From worker 6:	ClusterManagers
	From worker 4:	DSGE
	From worker 3:	BinDeps
	From worker 7:	CURAND
	From worker 5:	FixedSizeArrays
	From worker 3:	Bio
	From worker 7:	Currencies
	From worker 6:	CmplxRoots
	From worker 5:	FixedSizeDictionaries
	From worker 6:	Codecs
	From worker 4:	DSP
	From worker 4:	DualNumbers
	From worker 3:	BiomolecularStructures
	From worker 6:	CodeTools
	From worker 7:	CurveFit
	From worker 5:	Fixtures
	From worker 4:	DustExtinction
	From worker 2:	Accumulo
	From worker 2:	ACME
	From worker 3:	BioSeq
	From worker 6:	CoinOptServices
	From worker 5:	FLAC
	From worker 7:	CUSOLVER
	From worker 4:	DWARF
	From worker 2:	ActiveAppearanceModels
	From worker 2:	Actors
	From worker 6:	C

### Problems

* Parsing hangs after a certain point. Investigating why
    * Failing indexes `292 303 302 305 306 304`
* Seemed like it hung after being mostly finished this time. I think I will try to do a more efficient search this time (i.e. only look in module files.)

In [ ]:
map(x->DataOps.module_usage(repos[292,:dir], x), pkgs)